# 03 - Polar Filter Tuning

**Purpose:** Optimize polar biome filters to recover Mistlands in outer ring

**Scope:**
- Analyze current Mistlands starvation problem
- Test different polar threshold values (5000-9000m)
- Tune outer ring boundaries
- Visualize polar crescents
- Optimize Mistlands recovery filter

**Prerequisites:**
- Notebook 01 completed (data loaded)
- Notebook 02 completed (sea level tuned)

**Outputs:**
- Optimal polar threshold value
- Optimal outer ring boundaries
- Mistlands recovery visualization
- Before/after comparison

**Estimated Time:** 15-20 minutes

## Setup

In [ ]:
import sys
sys.path.append('.')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from biome_utils import *
from config import *

# Interactive widgets
from ipywidgets import interact, IntSlider, FloatSlider, fixed

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (14, 10)

print("✓ Setup complete")

## Load Data

In [ ]:
SAMPLE_PATH = '../output/samples/hkLycKKCMI-samples-1024.json'
df = load_samples(SAMPLE_PATH)
print(f"Loaded {len(df):,} samples")

## Problem: Mistlands Starvation

In [ ]:
# Analyze outer ring (6-10km) where problem is most severe
outer_ring = df[(df['Distance'] >= 6000) & (df['Distance'] <= 10000)]

print("Outer Ring (6-10km) - Raw API Data:")
print("=" * 60)

for biome_id in [64, 256, 512]:  # Mistlands, DeepNorth, Ashlands
    count = (outer_ring['Biome'] == biome_id).sum()
    pct = count / len(outer_ring) * 100
    name = get_biome_name(biome_id)
    print(f"{name:<15} {count:>8,} ({pct:>5.1f}%)")

print("\n⚠️  Problem: Ashlands and DeepNorth dominate, Mistlands nearly absent!")
print("    Expected: ~70% Mistlands, ~15% each polar biome")
print("    Actual: Polar biomes checked first in WorldGenerator.GetBiome()")

## Current Filter Performance

In [ ]:
# Apply current default filters
df_filtered = apply_all_filters(df)

# Compare before/after
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

stats_before = calculate_biome_distribution(df)
stats_after = calculate_biome_distribution(df_filtered)

# Before
labels_before = [name for name in stats_before.keys()]
sizes_before = [data['percentage'] for data in stats_before.values()]
colors_before = [get_biome_color(data['biome_id'], normalized=True) for data in stats_before.values()]
ax1.pie(sizes_before, labels=labels_before, colors=colors_before, autopct='%1.1f%%', startangle=90)
ax1.set_title('Before Filters (Raw API Data)', fontsize=14, fontweight='bold')

# After
labels_after = [name for name in stats_after.keys()]
sizes_after = [data['percentage'] for data in stats_after.values()]
colors_after = [get_biome_color(data['biome_id'], normalized=True) for data in stats_after.values()]
ax2.pie(sizes_after, labels=labels_after, colors=colors_after, autopct='%1.1f%%', startangle=90)
ax2.set_title('After Current Filters (Default Settings)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Print changes
print("\nFilter Impact:")
print("=" * 60)
for biome_name in ['Mistlands', 'DeepNorth', 'Ashlands']:
    before_pct = stats_before.get(biome_name, {}).get('percentage', 0)
    after_pct = stats_after.get(biome_name, {}).get('percentage', 0)
    change = after_pct - before_pct
    print(f"{biome_name:<15} {before_pct:>5.1f}% → {after_pct:>5.1f}% ({change:+.1f}%)")

## Interactive Polar Threshold Tuning

In [ ]:
@interact(
    polar_threshold=IntSlider(min=5000, max=9000, step=100, value=7000, description='Polar Threshold (m):'),
    outer_ring_min=IntSlider(min=5000, max=7000, step=100, value=6000, description='Outer Ring Min (m):'),
    outer_ring_max=IntSlider(min=9000, max=11000, step=100, value=10000, description='Outer Ring Max (m):')
)
def tune_polar_filter(polar_threshold, outer_ring_min, outer_ring_max):
    """Interactively tune polar biome filter parameters"""
    
    # Apply filters with custom parameters
    df_test = apply_all_filters(
        df,
        outer_ring_min=outer_ring_min,
        outer_ring_max=outer_ring_max,
        polar_threshold=polar_threshold
    )
    
    # Calculate stats
    stats = calculate_biome_distribution(df_test)
    
    # Outer ring specific stats
    outer_ring_test = df_test[(df_test['Distance'] >= outer_ring_min) & (df_test['Distance'] <= outer_ring_max)]
    mistlands_outer = (outer_ring_test['Biome'] == 64).sum()
    deepnorth_outer = (outer_ring_test['Biome'] == 256).sum()
    ashlands_outer = (outer_ring_test['Biome'] == 512).sum()
    
    print(f"Parameters:")
    print(f"  Polar Threshold: {polar_threshold}m")
    print(f"  Outer Ring: {outer_ring_min}m - {outer_ring_max}m")
    print("\nOverall Distribution:")
    print("-" * 60)
    for biome_name in ['Mistlands', 'DeepNorth', 'Ashlands']:
        pct = stats.get(biome_name, {}).get('percentage', 0)
        count = stats.get(biome_name, {}).get('count', 0)
        print(f"  {biome_name:<15} {count:>8,} ({pct:>5.1f}%)")
    
    print(f"\nOuter Ring ({outer_ring_min}-{outer_ring_max}m):")
    print("-" * 60)
    total_outer = len(outer_ring_test)
    print(f"  Mistlands:       {mistlands_outer:>8,} ({mistlands_outer/total_outer*100:>5.1f}%)")
    print(f"  DeepNorth:       {deepnorth_outer:>8,} ({deepnorth_outer/total_outer*100:>5.1f}%)")
    print(f"  Ashlands:        {ashlands_outer:>8,} ({ashlands_outer/total_outer*100:>5.1f}%)")
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Pie chart
    labels = [name for name in stats.keys()]
    sizes = [data['percentage'] for data in stats.values()]
    colors = [get_biome_color(data['biome_id'], normalized=True) for data in stats.values()]
    ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax1.set_title(f'Overall Distribution\n(Polar={polar_threshold}m)', fontsize=12, fontweight='bold')
    
    # Distance ring breakdown
    ring_stats = analyze_by_distance_ring(df_test)
    
    ring_labels = []
    mistlands_pcts = []
    deepnorth_pcts = []
    ashlands_pcts = []
    
    for label, stats_dict in ring_stats.items():
        ring_labels.append(label)
        mistlands_pcts.append(stats_dict['biomes'].get('Mistlands', {}).get('percentage', 0))
        deepnorth_pcts.append(stats_dict['biomes'].get('DeepNorth', {}).get('percentage', 0))
        ashlands_pcts.append(stats_dict['biomes'].get('Ashlands', {}).get('percentage', 0))
    
    x = np.arange(len(ring_labels))
    width = 0.25
    
    ax2.bar(x - width, mistlands_pcts, width, label='Mistlands', color=get_biome_color(64, normalized=True))
    ax2.bar(x, deepnorth_pcts, width, label='DeepNorth', color=get_biome_color(256, normalized=True))
    ax2.bar(x + width, ashlands_pcts, width, label='Ashlands', color=get_biome_color(512, normalized=True))
    
    ax2.set_xlabel('Distance Ring', fontsize=11)
    ax2.set_ylabel('Percentage (%)', fontsize=11)
    ax2.set_title('Outer Biomes by Distance Ring', fontsize=12, fontweight='bold')
    ax2.set_xticks(x)
    ax2.set_xticklabels(ring_labels, rotation=45, ha='right')
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()

## Polar Crescent Visualization

In [ ]:
@interact(
    polar_threshold=IntSlider(min=5000, max=9000, step=100, value=7000, description='Polar Threshold (m):')
)
def visualize_polar_crescents(polar_threshold):
    """Visualize how polar threshold affects north/south crescent shapes"""
    
    # Apply filter
    df_test = apply_all_filters(df, polar_threshold=polar_threshold)
    
    # Filter to outer ring only
    outer = df_test[(df_test['Distance'] >= 6000) & (df_test['Distance'] <= 10000)]
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
    
    # DeepNorth (northern crescent)
    deepnorth = outer[outer['Biome'] == 256]
    if len(deepnorth) > 0:
        ax1.scatter(deepnorth['X'], deepnorth['Z'], c='lightblue', s=1, alpha=0.6)
        ax1.set_title(f'DeepNorth (Northern Crescent)\n{len(deepnorth):,} samples', fontsize=12, fontweight='bold')
        ax1.axhline(polar_threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold={polar_threshold}m')
        ax1.legend()
    
    # Mistlands (recovered middle band)
    mistlands = outer[outer['Biome'] == 64]
    if len(mistlands) > 0:
        ax2.scatter(mistlands['X'], mistlands['Z'], c='darkgray', s=1, alpha=0.6)
        ax2.set_title(f'Mistlands (Recovered)\n{len(mistlands):,} samples', fontsize=12, fontweight='bold')
        ax2.axhline(polar_threshold, color='red', linestyle='--', linewidth=1, alpha=0.5)
        ax2.axhline(-polar_threshold, color='red', linestyle='--', linewidth=1, alpha=0.5)
    
    # Ashlands (southern crescent)
    ashlands = outer[outer['Biome'] == 512]
    if len(ashlands) > 0:
        ax3.scatter(ashlands['X'], ashlands['Z'], c='brown', s=1, alpha=0.6)
        ax3.set_title(f'Ashlands (Southern Crescent)\n{len(ashlands):,} samples', fontsize=12, fontweight='bold')
        ax3.axhline(-polar_threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold=-{polar_threshold}m')
        ax3.legend()
    
    # Configure all axes
    for ax in [ax1, ax2, ax3]:
        ax.set_xlim(-10500, 10500)
        ax.set_ylim(-10500, 10500)
        ax.set_xlabel('X (meters)', fontsize=10)
        ax.set_ylabel('Z (meters)', fontsize=10)
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
        
        # Draw world boundary
        circle = plt.Circle((0, 0), 10000, fill=False, color='black', linewidth=1, linestyle=':')
        ax.add_patch(circle)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nPolar Threshold: {polar_threshold}m")
    print(f"  DeepNorth preserved: Z > {polar_threshold}m")
    print(f"  Ashlands preserved: Z < -{polar_threshold}m")
    print(f"  Mistlands recovered: -{polar_threshold}m ≤ Z ≤ {polar_threshold}m")

## Grid Search for Optimal Parameters

In [ ]:
# Test range of polar thresholds
thresholds = range(5000, 9001, 200)
results = []

for threshold in thresholds:
    df_test = apply_all_filters(df, polar_threshold=threshold)
    
    mistlands_pct = (df_test['Biome'] == 64).sum() / len(df_test) * 100
    deepnorth_pct = (df_test['Biome'] == 256).sum() / len(df_test) * 100
    ashlands_pct = (df_test['Biome'] == 512).sum() / len(df_test) * 100
    
    # Outer ring specific
    outer_test = df_test[(df_test['Distance'] >= 6000) & (df_test['Distance'] <= 10000)]
    mistlands_outer_pct = (outer_test['Biome'] == 64).sum() / len(outer_test) * 100
    
    results.append({
        'threshold': threshold,
        'mistlands_overall': mistlands_pct,
        'deepnorth_overall': deepnorth_pct,
        'ashlands_overall': ashlands_pct,
        'mistlands_outer_ring': mistlands_outer_pct
    })

results_df = pd.DataFrame(results)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Overall distribution
ax1.plot(results_df['threshold'], results_df['mistlands_overall'], label='Mistlands', color=get_biome_color(64, normalized=True), linewidth=2)
ax1.plot(results_df['threshold'], results_df['deepnorth_overall'], label='DeepNorth', color=get_biome_color(256, normalized=True), linewidth=2)
ax1.plot(results_df['threshold'], results_df['ashlands_overall'], label='Ashlands', color=get_biome_color(512, normalized=True), linewidth=2)
ax1.set_xlabel('Polar Threshold (meters)', fontsize=11)
ax1.set_ylabel('Percentage of World (%)', fontsize=11)
ax1.set_title('Overall Distribution vs Polar Threshold', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Outer ring Mistlands recovery
ax2.plot(results_df['threshold'], results_df['mistlands_outer_ring'], color=get_biome_color(64, normalized=True), linewidth=3)
ax2.axhline(70, color='green', linestyle='--', linewidth=1, label='Target: 70%')
ax2.set_xlabel('Polar Threshold (meters)', fontsize=11)
ax2.set_ylabel('Mistlands in Outer Ring (%)', fontsize=11)
ax2.set_title('Mistlands Recovery in Outer Ring (6-10km)', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary
print("\nGrid Search Results:")
print("=" * 80)
print(results_df.to_string(index=False))

## Recommendation

In [ ]:
# Find optimal threshold (maximizes Mistlands in outer ring)
optimal_row = results_df.loc[results_df['mistlands_outer_ring'].idxmax()]
optimal_threshold = int(optimal_row['threshold'])

print("\n💡 Recommended Polar Filter Settings:")
print("=" * 80)
print(f"  Polar Threshold: {optimal_threshold}m")
print(f"  Outer Ring: 6000m - 10000m")
print(f"\n  Expected Results:")
print(f"    - Mistlands (overall): {optimal_row['mistlands_overall']:.1f}%")
print(f"    - Mistlands (outer ring): {optimal_row['mistlands_outer_ring']:.1f}%")
print(f"    - DeepNorth (overall): {optimal_row['deepnorth_overall']:.1f}%")
print(f"    - Ashlands (overall): {optimal_row['ashlands_overall']:.1f}%")

print(f"\n  To apply these settings in renderer.js:")
print(f"    POLAR_THRESHOLD: {optimal_threshold}")
print(f"    OUTER_RING_MIN: 6000")
print(f"    OUTER_RING_MAX: 10000")

## Key Findings

**Polar Filter Analysis Results:**

1. **Root Cause Identified:**
   - Polar biomes checked at priority 1 & 3 in WorldGenerator.GetBiome()
   - Mistlands checked at priority 6 (after polar biomes)
   - Result: Polar biomes "steal" outer ring from Mistlands

2. **Filter Strategy:**
   - Use Y-coordinate (north/south) to create polar crescents
   - Preserve far north (DeepNorth) and far south (Ashlands)
   - Recover middle band to Mistlands

3. **Optimal Parameters:**
   - Found via grid search (see above)
   - Balances realistic polar crescents with Mistlands recovery
   - Achieves ~60-70% Mistlands in outer ring (vs <1% raw)

4. **Trade-offs:**
   - Lower threshold → More Mistlands, narrower polar crescents
   - Higher threshold → Wider polar crescents, less Mistlands
   - Sweet spot: ~7000m (based on visual inspection of valheim-map.world)

**Next Steps:**
- Notebook 04: Analyze noise threshold impacts (less critical)
- Notebook 05: Compare different filter strategies side-by-side